In [1]:
#Code adapted from DODUO: https://github.com/megagonlabs/doduo
#Define which GPU to use
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '6'

In [2]:
import argparse
import json
import math
import random
from time import time

import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
import torch
from torch.nn import CrossEntropyLoss, BCEWithLogitsLoss
from torch.utils.data import DataLoader, RandomSampler
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig, RobertaConfig, RobertaTokenizer, RobertaForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup

In [3]:
from dataset import (
    collate_fn,
    #CTA
    CTASingleColumnDataset,
    CTAAllTableDataset,
    #CPA
    CPASingleColumnDataset,
    CPAAllTableDataset,
)
from model import BertForMultiOutputClassification, BertMultiPairPooler
from util import f1_score_multilabel, set_seed

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
#Define the method to evaluate
#Possible method: cta, cpa or doduo
method = 'doduo'
#Specify which serialization strategy to use
#Possible: single-column, all-table ... (+ new ones)
serialization = 'single-column'
#Specify which language model to use:
#bert-base-uncased, roberta-base ...
model_name = 'bert-base-uncased'
#Doduo can be used only with BERT and all-table serialization
#RoBERTa can not be used with all-table serialization

In [6]:
if method == 'cta':
    tasks = ['cta']
elif method == 'cpa':
    tasks = ['cpa']
else: #doduo
    tasks = ['cta', 'cpa']

In [7]:
#Model parameters
max_length = 32
batch_size = 16
num_train_epochs = 5
#Number of classes per task
task_num_class_dict = {
        "cta": 91,
        "cpa": 176
    }
filepaths_task_dict = {
    "cta": "data/CTA/cta_lm.pkl",
    "cpa": "data/CPA/cpa_lm.pkl",
}
serialization_method_dict = {
    "cta": {
        "single-column": CTASingleColumnDataset,
        "all-table": CTAAllTableDataset
    },
    "cpa": {
        "single-column": CPASingleColumnDataset,
        "all-table": CPAAllTableDataset
    }
}

In [8]:
if not os.path.exists('model/'):
    print("{} not exists. Created".format(dirpath))
    os.makedirs(dirpath)

In [9]:
#Tokenizer based on language model
if 'roberta' in model_name:
    tokenizer = RobertaTokenizer.from_pretrained(model_name, use_fast=True)
    base_model = 'roberta'
else:
    tokenizer = BertTokenizer.from_pretrained(model_name, use_fast=True)
    base_model = 'bert'

In [10]:
#If Doduo there are two models: one for CTA and one for CPA
models = []
train_datasets = []
train_dataloaders = []
valid_datasets = []
valid_dataloaders = []

In [11]:
for task in tasks:
    if serialization == 'single-column':
        #Choose model
        if 'roberta' in model_name:
            model_config = RobertaConfig.from_pretrained(model_name, num_labels=task_num_class_dict[task])
            model = RobertaForSequenceClassification(model_config)
        else:
            model_config = BertConfig.from_pretrained(model_name, num_labels=task_num_class_dict[task])
            model = BertForSequenceClassification(model_config)
        
        #Choose serialization
        dataset_serialization = serialization_method_dict[task][serialization]
        
    #Add more conditions when adding new serialization methods        
    else:
        if 'roberta' in model_name:
            model = RobertaForMultiOutputClassification.from_pretrained(
                    model_name,
                    num_labels=task_num_class_dict[task],
                    output_attentions=False,
                    output_hidden_states=False,
                )
        else:
            model = BertForMultiOutputClassification.from_pretrained(
                    model_name,
                    num_labels=task_num_class_dict[task],
                    output_attentions=False,
                    output_hidden_states=False,
                )
        
        dataset_serialization = serialization_method_dict[task][serialization]
            
        #Doduo:
        if task == "cpa":
            # Use column pair embeddings
            config = BertConfig.from_pretrained(model_name)
            model.bert.pooler = BertMultiPairPooler(config).to(device)
            
            
    #Load training and validation datasets and datasetloaders
    train_dataset = dataset_serialization(filepath=filepaths_task_dict[task],
                                    split="train",
                                    tokenizer=tokenizer,
                                    max_length=max_length,
                                    bert=base_model,
                                    device=device)

    valid_dataset = dataset_serialization(filepath=filepaths_task_dict[task],
                                split="dev",
                                tokenizer=tokenizer,
                                max_length=max_length,
                                bert=base_model,
                                device=device)

    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset,
                                  sampler=train_sampler,
                                  batch_size=batch_size,
                                  collate_fn=collate_fn)
    valid_dataloader = DataLoader(valid_dataset,
                                  batch_size=batch_size,
                                  collate_fn=collate_fn)

    train_datasets.append(train_dataset)
    train_dataloaders.append(train_dataloader)
    valid_datasets.append(valid_dataset)
    valid_dataloaders.append(valid_dataloader)

    models.append(model.to(device))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultiOutputClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultiOutputClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiOutputClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiOutputClassification were not initialized from the model c

Loading already processed train dataset
Loading already processed dev dataset


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultiOutputClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultiOutputClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiOutputClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiOutputClassification were not initialized from the model c

Use column-pair pooling
Loading already processed train dataset
Loading already processed dev dataset


In [12]:
optimizers = []
schedulers = []
loss_fns = []

In [13]:
for i, train_dataloader in enumerate(train_dataloaders):
    t_total = len(train_dataloader) * num_train_epochs
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [
                p for n, p in models[i].named_parameters()
                if not any(nd in n for nd in no_decay)
            ],
            "weight_decay":
            0.0
        },
        {
            "params": [
                p for n, p in models[i].named_parameters()
                if any(nd in n for nd in no_decay)
            ],
            "weight_decay":
            0.0
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5, eps=1e-8)
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0,
                                                num_training_steps=t_total)
    optimizers.append(optimizer)
    schedulers.append(scheduler)
    loss_fns.append(CrossEntropyLoss())

set_seed(42)

In [14]:
best_vl_micro_f1s = [-1 for _ in range(len(tasks))]
best_vl_macro_f1s = [-1 for _ in range(len(tasks))]
epoch_evaluation_results = [[] for _ in range(len(tasks))]

In [15]:
from pdb import set_trace

In [16]:
for epoch in range(num_train_epochs):
    for task_number, (task, model, train_dataset, valid_dataset, train_dataloader,
            valid_dataloader, optimizer, scheduler, loss_fn,
            epoch_evaluation_result) in enumerate(
                zip(tasks, models, train_datasets, valid_datasets,
                    train_dataloaders, valid_dataloaders, optimizers,
                    schedulers, loss_fns, epoch_evaluation_results)):
        t1 = time()

        model.train()
        
        training_loss = 0.
        training_predictions = []
        training_labels = []
        
        validation_loss = 0.
        validation_predictions = []
        validation_labels = []

        for batch_idx, batch in enumerate(train_dataloader):
            
            if serialization == 'single-column':
                
                #Retrive input ids, attention masks and labels for batch
                batch_input_ids = batch["data"].T.to(device)
                batch_mask = batch["attention"].T.to(device)
                #For cross-entropy loss labels should not be vectors
                batch_labels = torch.tensor([label.tolist().index(1) for label in batch["label"]]).to(device)
                loss, logits = model(batch_input_ids, token_type_ids=None, attention_mask=batch_mask, labels=batch_labels, return_dict=False)

                #Retrive predicted labels for batch
                for pred in logits.argmax(axis=-1):
                    y = [0] * logits.shape[1]
                    y[pred] = 1
                    training_predictions.append(y)

                #True labels:
                training_labels += batch["label"].cpu().detach().numpy().tolist()
                
            else:
                # Table serialization case
                batch_labels = torch.tensor([label.tolist().index(1) for label in batch["label"] if 1 in label.tolist()]).to(device)
                logits, = model(input_ids = batch["data"].T)

                # Align the tensor shape when the size is 1
                if len(logits.shape) == 2:
                    logits = logits.unsqueeze(0)
                
                cls_indexes = torch.nonzero( batch["data"].T == tokenizer.cls_token_id)
                filtered_logits = torch.zeros(cls_indexes.shape[0], logits.shape[2]).to(device)
                
                #Mark where CLS tokens are located
                for n in range(cls_indexes.shape[0]):
                    i, j = cls_indexes[n]
                    logit_n = logits[i, j, :]
                    filtered_logits[n] = logit_n
                    
                if task == 'cta':
                    for pred in filtered_logits.argmax(axis=-1):
                        y = [0] * filtered_logits.shape[1]
                        y[pred] = 1
                        training_predictions.append(y)
                        
                    training_labels += batch["label"].cpu().detach().numpy().tolist()
                    loss = loss_fn(filtered_logits, batch_labels)
                    
                else:
                    #Change
                    all_preds = []
                    for pred in filtered_logits.argmax(axis=-1):
                        y = [0] * filtered_logits.shape[1]
                        y[pred] = 1
                        all_preds.append(y)
                    
                    all_labels = batch["label"].cpu().detach().numpy()
                    # Ignore the very first CLS token
                    idxes = np.where(all_labels > 0)[0]
                    #set_trace()
                
                    all_preds_filtered = [ pred for i, pred in enumerate(all_preds) if i in idxes ]
                    all_labels_filtered = [label.tolist() for label in batch["label"] if 1 in label.tolist()]
                    
                    training_predictions += all_preds_filtered
                    #set_trace()
                    training_labels += all_labels_filtered  
                    loss = loss_fn(filtered_logits, batch["label"].float())

            # Perform a backward pass to calculate the gradients.
            loss.backward()
            # Accumulate the training loss over all of the batches
            training_loss += loss.item()
            
            #Update parameters and take a step using the calculated gradient
            optimizer.step()
            #Update learning rate
            scheduler.step()
            #Clear previously calculated gradients
            model.zero_grad()

        training_loss /= (len(train_dataset) / batch_size)

        tr_micro_f1, tr_macro_f1, tr_class_f1, _ = f1_score_multilabel(training_labels, training_predictions)

        # Validation
        model.eval()
        for batch_idx, batch in enumerate(valid_dataloader):
            if serialization == 'single-column':
                                
                batch_input_ids = batch["data"].T.to(device)
                batch_mask = batch["attention"].T.to(device)
                #For cross-entropy loss labels should not be vectors
                batch_labels = torch.tensor([label.tolist().index(1) for label in batch["label"]]).to(device)

                loss, logits = model(batch_input_ids, token_type_ids=None, attention_mask=batch_mask, labels=batch_labels, return_dict=False)

                for p in logits.argmax(axis=-1):
                    y = [0] * logits.shape[1]
                    y[p] = 1
                    validation_predictions.append(y)

                validation_labels += batch["label"].cpu().detach().numpy().tolist()
                
            else:
                batch_labels = torch.tensor([label.tolist().index(1) for label in batch["label"] if 1 in label.tolist()]).to(device)
                logits, = model(input_ids = batch["data"].T)

                # Align the tensor shape when the size is 1
                if len(logits.shape) == 2:
                    logits = logits.unsqueeze(0)
                
                cls_indexes = torch.nonzero( batch["data"].T == tokenizer.cls_token_id)
                filtered_logits = torch.zeros(cls_indexes.shape[0], logits.shape[2]).to(device)
                
                #Mark where CLS tokens are located
                for n in range(cls_indexes.shape[0]):
                    i, j = cls_indexes[n]
                    logit_n = logits[i, j, :]
                    filtered_logits[n] = logit_n
                    
                if task == 'cta':
                    for pred in filtered_logits.argmax(axis=-1):
                        y = [0] * filtered_logits.shape[1]
                        y[pred] = 1
                        validation_predictions.append(y)
                        
                    validation_labels += batch["label"].cpu().detach().numpy().tolist()
                    loss = loss_fn(filtered_logits, batch_labels)
                    
                else:
                    all_preds = []
                    for pred in filtered_logits.argmax(axis=-1):
                        y = [0] * filtered_logits.shape[1]
                        y[pred] = 1
                        all_preds.append(y)

                    all_labels = batch["label"].cpu().detach().numpy()
                    # Ignore the very first CLS token
                    idxes = np.where(all_labels > 0)[0]
                
                    all_preds_filtered = [ pred for i, pred in enumerate(all_preds) if i in idxes ]
                    all_labels_filtered = [label.tolist() for label in batch["label"] if 1 in label.tolist()]
                    validation_predictions += all_preds_filtered
                    
                    validation_labels += all_labels_filtered 
                    loss = loss_fn(filtered_logits, batch["label"].float())
                
            validation_loss += loss.item()

        validation_loss /= (len(valid_dataset) / batch_size)

        vl_micro_f1, vl_macro_f1, vl_class_f1, _ = f1_score_multilabel(validation_labels, validation_predictions)
        
        #Mark highest micro-F1 and save model if it outputs highest F1
        if vl_micro_f1 > best_vl_micro_f1s[task_number]:
            
            best_vl_micro_f1s[task_number] = vl_micro_f1
            model_savepath = "model/{}_{}_{}_{}-bs{}-ml-{}.pt".format(method, task, serialization, model_name, batch_size, max_length)    
            torch.save(model.state_dict(), model_savepath)
            
        epoch_evaluation_result.append([training_loss, tr_macro_f1, tr_micro_f1, validation_loss, vl_macro_f1, vl_micro_f1])
            
        t2 = time()
            
        print(
        "Epoch {} ({}): tr_loss={:.7f} tr_macro_f1={:.4f} tr_micro_f1={:.4f} "
        .format(epoch, task, training_loss, tr_macro_f1, tr_micro_f1),
        "vl_loss={:.7f} vl_macro_f1={:.4f} vl_micro_f1={:.4f} (Total time: {:.2f} sec.)"
        .format(validation_loss, vl_macro_f1, vl_micro_f1, (t2 - t1)))
        

/ceph/kkorini/team-project/util.py:19: RuntimeWarning: invalid value encountered in true_divide
  class_r = conf_mat[:, 1, 1] /  conf_mat[:, :, 1].sum(axis=1)
/ceph/kkorini/team-project/util.py:19: RuntimeWarning: invalid value encountered in true_divide
  class_r = conf_mat[:, 1, 1] /  conf_mat[:, :, 1].sum(axis=1)


Epoch 0 (cta): tr_loss=0.9917576 tr_macro_f1=0.6761 tr_micro_f1=0.7589  vl_loss=0.5147462 vl_macro_f1=0.8144 vl_micro_f1=0.8658 (Total time: 636.12 sec.)


/ceph/kkorini/team-project/util.py:19: RuntimeWarning: invalid value encountered in true_divide
  class_r = conf_mat[:, 1, 1] /  conf_mat[:, :, 1].sum(axis=1)
/ceph/kkorini/team-project/util.py:19: RuntimeWarning: invalid value encountered in true_divide
  class_r = conf_mat[:, 1, 1] /  conf_mat[:, :, 1].sum(axis=1)


Epoch 0 (cpa): tr_loss=1.0363383 tr_macro_f1=0.5251 tr_micro_f1=0.6425  vl_loss=0.5396246 vl_macro_f1=0.7158 vl_micro_f1=0.7984 (Total time: 680.11 sec.)
Epoch 1 (cta): tr_loss=0.3535146 tr_macro_f1=0.8671 tr_micro_f1=0.8973  vl_loss=0.4094755 vl_macro_f1=0.8612 vl_micro_f1=0.8889 (Total time: 618.50 sec.)


/ceph/kkorini/team-project/util.py:19: RuntimeWarning: invalid value encountered in true_divide
  class_r = conf_mat[:, 1, 1] /  conf_mat[:, :, 1].sum(axis=1)
/ceph/kkorini/team-project/util.py:19: RuntimeWarning: invalid value encountered in true_divide
  class_r = conf_mat[:, 1, 1] /  conf_mat[:, :, 1].sum(axis=1)


Epoch 1 (cpa): tr_loss=0.3507748 tr_macro_f1=0.7985 tr_micro_f1=0.8532  vl_loss=0.4035558 vl_macro_f1=0.7863 vl_micro_f1=0.8393 (Total time: 675.62 sec.)
Epoch 2 (cta): tr_loss=0.2214538 tr_macro_f1=0.9174 tr_micro_f1=0.9319  vl_loss=0.4172381 vl_macro_f1=0.8654 vl_micro_f1=0.8907 (Total time: 613.47 sec.)
Epoch 2 (cpa): tr_loss=0.2314727 tr_macro_f1=0.8666 tr_micro_f1=0.8975  vl_loss=0.3682750 vl_macro_f1=0.8087 vl_micro_f1=0.8517 (Total time: 673.11 sec.)
Epoch 3 (cta): tr_loss=0.1418599 tr_macro_f1=0.9483 tr_micro_f1=0.9541  vl_loss=0.3939214 vl_macro_f1=0.8793 vl_micro_f1=0.9014 (Total time: 613.92 sec.)
Epoch 3 (cpa): tr_loss=0.1623803 tr_macro_f1=0.9099 tr_micro_f1=0.9269  vl_loss=0.3622168 vl_macro_f1=0.8196 vl_micro_f1=0.8603 (Total time: 678.29 sec.)
Epoch 4 (cta): tr_loss=0.0885947 tr_macro_f1=0.9710 tr_micro_f1=0.9725  vl_loss=0.4003124 vl_macro_f1=0.8816 vl_micro_f1=0.9052 (Total time: 617.00 sec.)
Epoch 4 (cpa): tr_loss=0.1154948 tr_macro_f1=0.9403 tr_micro_f1=0.9500  vl_l

In [17]:
for task, epoch_evaluation in zip(tasks, epoch_evaluation_results):
    loss_info_df = pd.DataFrame(epoch_evaluation,
                                columns=[
                                    "tr_loss", "tr_f1_macro_f1",
                                    "tr_f1_micro_f1", "vl_loss",
                                    "vl_f1_macro_f1", "vl_f1_micro_f1"
                                ])
    
    loss_info_df.to_csv("model/{}_{}_{}_{}-bs{}-ml-{}_info.csv".format(method, task, serialization, model_name, batch_size, max_length))